# Desafios idwall


## Resolução da Parte 1 do Desafio 2 - Crawlers

Web Scraping do Reddit, escrito em Python, utilizando as bibliotecas _request_ e _BeautifulSoup_.

#### Desafio:
Encontrar e listar as _threads_ com 5000 pontos ou mais no Reddit naquele momento.


### Resolução:

É possível realizar o scrapping via PRAW, um wrapper para o API do Reddit, o qual permite realizar scrapings dos subreddits, criar um bot, entre outras funcionalidades.

Seguem dois sites que ensinam como realizar scrapping por esse método:

https://towardsdatascience.com/scraping-reddit-data-1c0af3040768

http://www.storybench.org/how-to-scrape-reddit-with-python/


Mas aqui, com o intuito de demonstrar habilidades mais gerais, vamos realizar o scrapping utilizando os pacotes 'request' e 'BeautifulSoup'.

Para uma breve introdução sobre web scraping e aplicação destes pacotes ver:

https://www.scrapehero.com/a-beginners-guide-to-web-scraping-part-1-the-basics/

https://www.youtube.com/watch?v=ng2o98k983k&t=1428s


Dito isto, vamos começar pelo simples e buscar as _top threads_ dentro do subreddit 'r/AskReddit': https://www.reddit.com/r/AskReddit/top/?t=day

"Subreddits são como fóruns dentro do Reddit e as postagens são chamadas threads.

Para quem gosta de gatos, há o subreddit '/r/cats' com threads contendo fotos de gatos fofinhos. Para threads sobre o Brasil, vale a pena visitar '/r/brazil' ou ainda '/r/worldnews'. Um dos maiores subreddits é o '/r/AskReddit'."

### Abrindo a url e salvando o arquivo em html:

In [1]:
# Primeiramente, importam-se as bibliotecas necessárias para o scrapping:
import requests
from bs4 import BeautifulSoup

# E cria-se uma função para salvar e outra para
# abrir a página html, a fim de minimizar danos ao servidor:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

#### ATENÇÃO!!!

Recomenda-se não rodar o código da célula abaixo, sendo utilizado assim apenas para simples conferência.

Como o reddit possui um sistema automatizado que impede mais que um request a cada dois segundos é possível que o código abaixo gere um "erro", de forma a não ser capaz de obter o código html do site. Eu tentei procurar entender o porquê, mas não obtive uma resposta.

Mas caso queira rodar o código, é necessário tentar algumas vezes, caso não consiga de primeira, até conseguir obter o html.

In [2]:
# Para abrir localmente e trabalhar com o arquivo e não com vários requests:
html = open_html('askreddit_top_day')

soup = BeautifulSoup(html, 'lxml')

print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <script>
   var __SUPPORTS_TIMING_API = typeof performance === 'object' && !!performance.mark && !! performance.measure && !!performance.getEntriesByType;
          function __perfMark(name) { __SUPPORTS_TIMING_API && performance.mark(name); };
          var __firstLoaded = false;
          function __markFirstPostVisible() {
            if (__firstLoaded) { return; }
            __firstLoaded = true;
            __perfMark("first_post_title_image_loaded");
          }
  </script>
  <script>
   __perfMark('head_tag_start');
  </script>
  <title>
   Ask Reddit...
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="origin-when-cross-origin" name="referrer"/>
  <style>
   /* http://meyerweb.com/eric/tools/css/reset/
    v2.0 | 20110126
    License: none (public domain)
  */

  html, body, div, span, applet, object, iframe,
  h1, h2, h3, h4, h5, h6, p, blockquote, pre,
  a, 

### Realizando o parsing no código reddit:

Primeiramente vamos analisar se o primeiro top thread é o desejado:

In [3]:
## html do bloco completo contendo todos os comentários
all_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

## html da tag e classe do número de pontos de certa thread:
# <div class="_1rZYMD_4xY3gRcSS3p8ODO" style="color:#1A1A1B">22.8k</div>
pontos_thread = all_threads.find('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
pontos_primeira_thread = pontos_thread.text

## html da tag e classe do texto de uma certa thread
# <h3 class="_eYtD2XCVieq6emjKBH3m">Teachers of Reddit, what was the most obvious "teacher crush" someone had on you?</h3>
thread = all_threads.find('h3', class_="_eYtD2XCVieq6emjKBH3m")
texto_thread = thread.text

## html do link do texto
# <a data-click-id="body" class="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE" href="/r/AskReddit/comments/cyqtk2/teachers_of_reddit_what_was_the_most_obvious/"><div class="_2SdHzo12ISmrC8H86TgSCp _3wqmjmv3tb_k-PROt7qFZe " style="--posttitletextcolor:#444e59" theme="[object Object]"><h3 class="_eYtD2XCVieq6emjKBH3m">Teachers of Reddit, what was the most obvious "teacher crush" someone had on you?</h3></div></a>
link = all_threads.find('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")
referencia = link['href']
texto_link = f'https://www.reddit.com{referencia}'

In [4]:
# Resultado dos pontos e texto do primeiro top thread do dia:
print(pontos_primeira_thread)
print(texto_thread)
print(texto_link)

28.8k
Everyone has a scar on their body from something dumb, they did as a child. What's your story?
https://www.reddit.com/r/AskReddit/comments/cz2apy/everyone_has_a_scar_on_their_body_from_something/


Tudo certo até aqui, então vamos seguir com os próximos passos.

### Realizando um loop sobre todas as top threads desejadas:

In [5]:
table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

extracted_points = []
for points in all_points_thread:
    point = points.text
    extracted_points.append(point)

extracted_texts = []
for threads in all_texts:
    thread = threads.text
    extracted_texts.append(thread)
    
extracted_links = []
for links in all_links:
    referencia = links['href']
    if referencia.startswith('http'):
        extracted_links.append(referencia)
    else:
        texto_link = f'https://www.reddit.com{referencia}'
        extracted_links.append(texto_link)
        
print(len(extracted_points))
print(len(extracted_texts))

16
8


Nesse código, é possivel ver que a lista contendo as pontuações está duplicada. Provavelmente isto ocorre por haverem classes repitidas no código html. Para consertar isto, basta criarmos uma nova lista contendo os valores únicos:

In [6]:
print(extracted_points)
unique_extracted_points = []
for i in range(0, len(extracted_points), 2):
    unique_extracted_points.append(extracted_points[i])
    
print(unique_extracted_points)

['28.8k', '28.8k', '3.3k', '3.3k', '2.7k', '2.7k', '1.2k', '1.2k', '1.5k', '1.5k', '812', '812', '936', '936', '563', '563']
['28.8k', '3.3k', '2.7k', '1.2k', '1.5k', '812', '936', '563']


In [7]:
top_threads = []
for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
    if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
        likes = p[0:-1]
        likes = int(float(likes)*1000)
        if likes >= 5000:
            subreddit = l.split('/')[4]
            s = f'/r/{subreddit}'
            record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
            top_threads.append(record)
            
print(top_threads)

[{'pontuacao': '28.8k', 'subreddit': '/r/AskReddit', 'titulo thread': "Everyone has a scar on their body from something dumb, they did as a child. What's your story?", 'link para os comentarios': 'https://www.reddit.com/r/AskReddit/comments/cz2apy/everyone_has_a_scar_on_their_body_from_something/'}]


In [8]:
# Opcionalmente podemos salvar o resultando em um arquivo json ou csv para uso futuro

import json

with open('data.json', 'w') as outfile:
    json.dump(top_threads, outfile, indent=4)
    
import csv
csv_file = open('data.csv', 'w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['pontuacao', 'subreddit', 'thread', 'link'])

# inserir csv_writer.writerow([p, s , t , l]) no loop das "top_threads = []"

33

### Realizando o scrapping na pagina principal

Agora vamos ir para uma página mais geral do site www.reddit.com, e realizar um novo scrape (Novamente, a célula abaixo foi convertida em Raw NBConvert, para evitar rodá-la acidentalmente):

In [9]:
# Código restante:

html = open_html('reddit_top_today')

soup = BeautifulSoup(html, 'lxml')

table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

extracted_points = []
for points in all_points_thread:
    point = points.text
    extracted_points.append(point)

extracted_texts = []
for threads in all_texts:
    thread = threads.text
    extracted_texts.append(thread)
    
extracted_links = []
for links in all_links:
    referencia = links['href']
    if referencia.startswith('http'):
        extracted_links.append(referencia)
    else:
        texto_link = f'https://www.reddit.com{referencia}'
        extracted_links.append(texto_link)

unique_extracted_points = []
for i in range(0, len(extracted_points), 2):
    unique_extracted_points.append(extracted_points[i])

top_threads = []
for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
    if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
        likes = p[0:-1]
        likes = int(float(likes)*1000)
        if likes >= 5000:
            subreddit = l.split('/')[4]
            s = f'/r/{subreddit}'
            record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
            top_threads.append(record)
            csv_writer.writerow([p, s , t , l])

csv_file.close()
for i in range(len(top_threads)):
    print(top_threads[i])
    print()

{'pontuacao': '121k', 'subreddit': '/r/aww', 'titulo thread': 'Scared cat gets saved by two French guys', 'link para os comentarios': 'https://www.reddit.com/r/aww/comments/cyv97r/scared_cat_gets_saved_by_two_french_guys/'}

{'pontuacao': '111k', 'subreddit': '/r/memes', 'titulo thread': 'The Area 51 raid is still happening right?', 'link para os comentarios': 'https://www.reddit.com/r/memes/comments/cz2i20/the_area_51_raid_is_still_happening_right/'}

{'pontuacao': '106k', 'subreddit': '/r/pics', 'titulo thread': 'In 1964, Ringo Starr snapped a photo of some high school students who skipped class to see the Beatles during their first trip to the US. The group had no idea the photo existed until Ringo published his book of photos. Nearly 50 years later, the group reunited and recreated the photo.', 'link para os comentarios': 'https://www.reddit.com/r/pics/comments/cyx1os/in_1964_ringo_starr_snapped_a_photo_of_some_high/'}

{'pontuacao': '103k', 'subreddit': '/r/aww', 'titulo thread': 

### Realizando o scrapping em uma lista de subreddits

Agora que entendemos como realizar o scrappping em um único link, o próximo passo seria realizar o scraping a partir de uma lista de subreddits separados por ponto-e-vírgula, e.g., "programming;dogs;brazil".

In [11]:
# input: programming;dogs;brazil
subreddits = str(input('Quais subreddits gostaria de acompanhar hoje? \n'))
subreddits_separated = subreddits.split(';')

print('\n', subreddits_separated)

Quais subreddits gostaria de acompanhar hoje? 
programming;dogs;brazil

 ['programming', 'dogs', 'brazil']


In [12]:
# A partir da lista criada anteriormente, cria-se outra lista com as urls para download do código html
urls_subreddits = []
for i in range(len(subreddits_separated)):
    url_link = 'https://www.reddit.com/r/'+subreddits_separated[i]
    urls_subreddits.append(url_link)

print(urls_subreddits)

['https://www.reddit.com/r/programming', 'https://www.reddit.com/r/dogs', 'https://www.reddit.com/r/brazil']


Indo para a pasta principal, é possível ver que os arquivos foram salvos. Mas caso rodássemos o código acima, por conta da proteção que o site reddit possui, os htmls salvos não corresponderiam aos desejados. Por conta disto, foi necessário uma intervenção humana no looping para que fosse possível obter os htmls desejados, tendo que baixar um por vez.

A partir da lista de htmls é possível realizar o mesmo scrapping anterior utilizando um loop do código anterior, como mostrado abaixo: 

In [15]:
for i in subreddits_separated:
    
    html = open_html(f'reddit_{i}')

    soup = BeautifulSoup(html, 'lxml')

    table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

    all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
    all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
    all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

    # Nesses dois casos (extracted_points e extracted_texts), é possível criarmos uma função, 
    # mas aqui como achei que o código não possui uma recorrência alta desses loops
    # preferi deixá-los explícitos
    extracted_points = []
    for points in all_points_thread:
        point = points.text
        extracted_points.append(point)

    extracted_texts = []
    for threads in all_texts:
        thread = threads.text
        extracted_texts.append(thread) 

    extracted_links = []
    for links in all_links:
        referencia = links['href']
        if referencia.startswith('http'):
            extracted_links.append(referencia)
        else:
            texto_link = f'https://www.reddit.com{referencia}'
            extracted_links.append(texto_link)
    
    unique_extracted_points = []
    for j in range(0, len(extracted_points), 2):
        unique_extracted_points.append(extracted_points[j])

    top_threads = []
    for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
        if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
            likes = p[0:-1]
            likes = int(float(likes)*1000)
            if likes >= 5000:
                subreddit = l.split('/')[4]
                s = f'/r/{subreddit}'
                record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
                top_threads.append(record)

    print(f'Top threads de /r/{i}: ', top_threads)
    print()

Top threads de /r/programming:  []

Top threads de /r/dogs:  []

Top threads de /r/brazil:  []



Nos casos acima, não houveram threads com mais de 5000 pontos. Além disto, foi possível rodar o código por conta da "intervenção" comentada anteriormente.

Caso modificássemos o limite de 5000 pontos, para 500 pontos por exemplo, podemos obter:

In [22]:
for i in subreddits_separated:
    
    html = open_html(f'reddit_{i}')

    soup = BeautifulSoup(html, 'lxml')

    table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

    all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
    all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
    all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

    extracted_points = []
    for points in all_points_thread:
        point = points.text
        extracted_points.append(point)

    extracted_texts = []
    for threads in all_texts:
        thread = threads.text
        extracted_texts.append(thread)

    extracted_links = []
    for links in all_links:
        referencia = links['href']
        if referencia.startswith('http'):
            extracted_links.append(referencia)
        else:
            texto_link = f'https://www.reddit.com{referencia}'
            extracted_links.append(texto_link)

    unique_extracted_points = []
    for j in range(0, len(extracted_points), 2):
        unique_extracted_points.append(extracted_points[j])
    
    top_threads = []
    for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
        if len(p) > 1: # Note que a partir daqui, o código foi modificado com o fim de obter as threads com pontuação menor que 1k
            likes = p
            if p[-1] == 'k':
                likes = p[0:-1]
                likes = int(float(likes)*1000)
            else:
                likes = int(likes)
            if likes >= 500:
                subreddit = l.split('/')[4]
                s = f'/r/{subreddit}'
                record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
                top_threads.append(record)
    if top_threads == []:
        top_threads = "Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)"
    print(f'Top threads de /r/{i}: ', top_threads)
    print(unique_extracted_points)
    print()

Top threads de /r/programming:  [{'pontuacao': '2.1k', 'subreddit': '/r/programming', 'titulo thread': 'Former Google engineer breaks down interview problems he uses to screen candidates. Lots of good coding, algorithms, and interview tips.', 'link para os comentarios': 'https://www.reddit.com/r/programming/comments/cz6f5r/former_google_engineer_breaks_down_interview/'}]
['2.1k', '438', '62', '77', '14', '12', '10', '16']

Top threads de /r/dogs:  [{'pontuacao': '558', 'subreddit': '/r/dogs', 'titulo thread': '[RIP] Joey, beagle, 14 y.o.', 'link para os comentarios': 'https://www.reddit.com/r/dogs/comments/cz54pu/rip_joey_beagle_14_yo/'}, {'pontuacao': '999', 'subreddit': '/r/dogs', 'titulo thread': '[RIP] Tutya, pug, 4 y.o.', 'link para os comentarios': 'https://www.reddit.com/r/dogs/comments/cyst1s/rip_tutya_pug_4_yo/'}]
['4', '9', '558', '85', '42', '16', '999', '24']

Top threads de /r/brazil:  Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)
['20', '24', '

No caso acima, juntamente com os threads, também mandei imprimir os valores dos likes. Ao visitarmos as duas últimas  páginas ('r/dogs', 'r/brazil'), vemos que realmente não existem muitos pontos nos top diários. Assim, no caso em que não temos threads com o valor de pontos desejados, nosso código acaba não retornando valores/resultados para esses casos.

## Resolução extra - Função para 'reddit scrapping':
Por fim, podemos criar uma função que recebe os subreddits desejados e retorna as top threads:

In [39]:
def top_threads():
    import requests
    from bs4 import BeautifulSoup
    
    subreddits = str(input('Quais subreddits gostaria de acompanhar hoje? \n'))
    subreddits_separated = subreddits.split(';')
    
    urls_subreddits = []
    for i in range(len(subreddits_separated)):
        url_link = 'https://www.reddit.com/r/' + subreddits_separated[i]
        urls_subreddits.append(url_link)
    
#     Aqui salvaríamos os htmls usando um for loop:
#     for i in range(0, len(urls_subreddits)):
#         url = urls_subreddits[i]
#         codigo_html = requests.get(url)
#         save_html(codigo_html.content, f'reddit_{subreddits_separated[i]}')

    for i in subreddits_separated:

        html = open_html(f'reddit_{i}')

        soup = BeautifulSoup(html, 'lxml')

        table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

        all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
        all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
        all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

        extracted_points = []
        for points in all_points_thread:
            point = points.text
            extracted_points.append(point)

        extracted_texts = []
        for threads in all_texts:
            thread = threads.text
            extracted_texts.append(thread)

        extracted_links = []
        for links in all_links:
            referencia = links['href']
            if referencia.startswith('http'):
                extracted_links.append(referencia)
            else:
                texto_link = f'https://www.reddit.com{referencia}'
                extracted_links.append(texto_link)

        unique_extracted_points = []
        for j in range(0, len(extracted_points), 2):
            unique_extracted_points.append(extracted_points[j])

        top_threads = []
        for p, t, l in zip(unique_extracted_points, extracted_texts, extracted_links):
            if len(p) > 1: # Note que a partir daqui, o código foi modificado com o fim de obter as threads com pontuação menor que 1k
                likes = p
                if p[-1] == 'k':
                    likes = p[0:-1]
                    likes = int(float(likes)*1000)
                else:
                    likes = int(likes)
                if likes >= 5000:
                    subreddit = l.split('/')[4]
                    s = f'/r/{subreddit}'
                    record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
                    top_threads.append(record)
        if top_threads == []:
            top_threads = "Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)"
        print()
        print(f'Top threads de /r/{i}: ', top_threads)

In [40]:
# input: programming;dogs;brazil
top_threads()

Quais subreddits gostaria de acompanhar hoje? 
programming;dogs;brazil

Top threads de /r/programming:  Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)

Top threads de /r/dogs:  Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)

Top threads de /r/brazil:  Nao ha top threads com os requisitos desejados nesse dia. Volte amanha :)
